# Eden AI

Eden AI is an AI consulting company that was founded to use its resources to empower people and create impactful products that use AI to improve the quality of life for individuals, businesses and societies at large.

This example goes over how to use LangChain to interact with Eden AI models



## Calling a model


Find a feature and model  on the EdenAI website.



### text generation

In [1]:
from langchain.llms import EdenAI
from langchain import PromptTemplate, LLMChain

llm=EdenAI(feature="text" ,sub_feature="generation" , params={"providers": "openai","temperature" : 0.2,"max_tokens" : 250} )

prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""

llm(prompt)

" No, a dog cannot drive a car.\n\nReasoning: \n1. Driving a car requires a driver's license, which only humans can obtain. \n2. Driving a car requires the ability to understand and follow traffic laws, which only humans can do. \n3. Driving a car requires the ability to operate a vehicle, which only humans can do. \n4. Therefore, a dog cannot drive a car."

### image generation

In [2]:
import base64
from io import BytesIO
from PIL import Image
import json
def print_base64_image(base64_string):
    # Decode the base64 string into binary data
    decoded_data = base64.b64decode(base64_string)

    # Create an in-memory stream to read the binary data
    image_stream = BytesIO(decoded_data)

    # Open the image using PIL
    image = Image.open(image_stream)

    # Display the image
    image.show()

In [3]:
text2image = EdenAI(
    feature="image" ,sub_feature="generation" ,
    params={
    "providers": "openai",
    "text": "this is a test of Eden AI",
    "resolution" : "512x512"
    }
)

In [4]:
image_output = text2image("A cat riding a motorcycle by Picasso")

#to dict 

image=json.loads(image_output)

In [13]:
print_base64_image(image[0]["image"])

### text generation with callback

In [15]:
from langchain.llms import EdenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = EdenAI(
    callbacks=[StreamingStdOutCallbackHandler()],
    feature="text" ,sub_feature="generation" , params={"providers": "openai","temperature" : 0.2,"max_tokens" : 250}
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
print(llm(prompt))

 No, a dog cannot drive a car.

Reasoning: 

1. Driving a car requires a driver's license, which is issued to humans only. 
2. Dogs do not have the physical capability to operate a car. 
3. Dogs do not have the cognitive ability to understand the rules of the road and safely operate a car. 
4. Therefore, a dog cannot drive a car.


## Chaining Calls

In [6]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = EdenAI(
feature="text" ,sub_feature="generation" , params={"providers": "openai","temperature" : 0.2,"max_tokens" : 250}
)
text2image = EdenAI(
feature="image" ,sub_feature="generation" , params={"providers": "openai", "resolution" : "512x512"}
)

In [8]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a description of a logo for this company: {company_name}, the logo should not contain text at all ",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [10]:
third_prompt = PromptTemplate(
    input_variables=["company_logo_description"],
    template="{company_logo_description}",
)
chain_three = LLMChain(llm=text2image, prompt=third_prompt)

In [11]:
# Run the chain specifying only the input variable for the first chain.
overall_chain = SimpleSequentialChain(
    chains=[chain, chain_two, chain_three],verbose=True
)
catchphrase = overall_chain.run("hats")




> Entering new SimpleSequentialChain chain...




Hat Haven.


The logo for Hat Haven is a stylized illustration of a top hat. The hat is colored in a deep navy blue, with a bright yellow band around the base. The brim of the hat is slightly curved, giving it a classic, timeless look. The yellow band is decorated with a pattern of small stars, giving the logo a whimsical and fun feel.
[{"image": "...", "image_resource_url": "https://d14uq1pz7dzsdq.cloudfront.net/6968afd8-aa71-41d3-93f6-1733ba691458_.png?Expires=1691082440&Signature=Wi-PQYUxd9eQbwpMXn~tGK86-vrbevcp0NUBSwhptjqiKBjZN8ZJGNVhhW16QWDGpO0jzMe896Rwl5HAaaTZ8pbebgY2TuPTPohWakaldoyBU2s-tfePt0IVsDFzD51VgUCg-T~lro~SJhFC~ZbbbMMRodReCxQNd3apdz8FsBiOQNMaXqXzFWCGC2PvB6y-DkLTsx1Mk0x4VudQxY-0Lk8NtWicaJtXTs6P0PeOvejO-HvEZEEEf8rGSBcEkkEAV~BPzuSLYV50-5XqDWJdLONrC6ZqyqRNQpoI3gY-0Hua2cIVMf7BPx8Xcr4OYjlOIa8mHIJIyfSdR3lDbJTuhA__&Key-Pair-Id=K1F55BTI9AHGIK"}]

> Finished chain.


In [12]:
#str to dict
result= json.loads(catchphrase)

#print the image
print_base64_image(result[0]['image'])